# Agent Tracing - Learning Objectives

This notebook demonstrates how to implement and use tracing capabilities in AI agent applications. By the end of this notebook, you will learn:

1. **How to enable tracing for agent runs** - Setting up basic and advanced tracing configurations
2. **How to customize trace metadata** - Adding contextual information to your traces
3. **How to maintain trace continuity across multiple agent runs** - Using the same trace ID for related queries
4. **How to visualize agent execution flows** - Creating visual representations of agent behavior

## Why Use Tracing?

Tracing provides several important benefits for AI agent development and operation:

- **Debugging and Troubleshooting**: Trace the execution path of agents to identify issues in complex workflows
- **Performance Monitoring**: Track latency, token usage, and other metrics to optimize agent performance
- **Transparency**: Gain visibility into how agents make decisions and use tools
- **Auditability**: Create records of agent activities for compliance and governance purposes
- **User Experience Improvement**: Analyze patterns in agent behavior to improve prompts and tool designs

This notebook shows both simple tracing setups and more advanced configurations with custom metadata and visualization.

In [ ]:
# https://mlflow.org/docs/latest/tracing/integrations/openai-agent
# https://openai.github.io/openai-agents-python/tracing/

In [1]:
from dotenv import load_dotenv
from agents import (

    Agent,
    Runner,
    RunConfig,
    trace,
)


load_dotenv()

True

In [2]:
from helpers.model_client import (
    get_openai_client,
    get_github_model_provider,
)

client = get_openai_client()
GITHUB_MODEL_PROVIDER = get_github_model_provider(client, "gpt-4o-mini")


In [ ]:
from helpers.tool_functions import(
    get_current_time,
    get_weather,
)

agent = Agent(
    name="Assistant",
    instructions="Answer the user's questions.",
    tools=[get_weather, get_current_time],

)

In [4]:
# https://github.com/openai/openai-agents-python/blob/main/examples/model_providers/custom_example_provider.py


try:
    with trace(workflow_name="msdiagentic_hello_tracking"):
        result = await Runner.run(
            agent,
            "What is the weather in New York? And what is the time?",
            run_config=RunConfig(model_provider=GITHUB_MODEL_PROVIDER),

        )
        print(result.final_output)
except Exception as e:
    print(f"[error] {e}")


[debug-server] get_current_weather(New York)
[debug] get_current_time called with location: New York
### Weather in New York:
- **Condition:** Overcast
- **Temperature:** 6 °C
- **Wind:** 4 km/h
- **Precipitation:** 0.0 mm

### Current Time in New York:
- **Time:** 07:12 PM


In [5]:
from datetime import datetime


timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
tr = trace(
    workflow_name=f"agent_tracing_workflow_{timestamp}",
    group_id="mdsi",
    metadata={
        "key1": "value1",
        "key2": "value2",
    },
)

# print trace workflow name
print(f"Trace workflow name: {tr.name}")
tr.start()

trace_id = tr.trace_id
print(f"Trace ID: {trace_id}")

run_config = RunConfig(
    model_provider=GITHUB_MODEL_PROVIDER,
    trace_id=trace_id,  # Pass the trace ID to the run config
    trace_metadata={"AgentName": "Weather and Time Agent"},
    workflow_name=tr.name,  # Pass the workflow name to the run config
    group_id=tr.group_id,  # Pass the group ID to the run config
)


Trace workflow name: agent_tracing_workflow_20250406_191300
Trace ID: trace_e3487f8519694093a2658bc6323fe704


In [6]:
try:
    result = await Runner.run(
            agent,
            "What is the weather in New York? And time.",
            run_config=run_config,
        )

    print(result.final_output)
except Exception as e:
    print(f"[error] {e}")

[debug-server] get_current_weather(New York)
[debug] get_current_time called with location: New York
The current weather in New York is overcast with a temperature of about 6°C. The wind is blowing at 4 km/h, with no precipitation reported.

The current time in New York is 7:13 PM.


In [7]:
try:
    result = await Runner.run(
            agent,
            "What is the time in Sydney",
            run_config=run_config,
        )

    print(result.final_output)
except Exception as e:
    print(f"[error] {e}")

[debug] get_current_time called with location: Sydney
The current time in Sydney is 07:13 PM.


In [8]:
try:
    result = await Runner.run(
            agent,
            "Tell me a joke!",
            run_config=run_config,
        )

    print(result.final_output)
except Exception as e:
    print(f"[error] {e}")

Why did the scarecrow win an award?

Because he was outstanding in his field!


In [9]:
tr.finish()

In [10]:
tracing_url = f"https://platform.openai.com/traces/trace?trace_id={trace_id}"

print(f"Tracing URL: {tracing_url}")

Tracing URL: https://platform.openai.com/traces/trace?trace_id=trace_e3487f8519694093a2658bc6323fe704


View Traces [here](https://platform.openai.com/traces?group_id=Dummy+Group)

# Add Viz

requires GraphViz extension to view

In [11]:
from agents.extensions.visualization import draw_graph

try:
    draw_graph(agent, filename="viz/05_agent_tracing.gv")
except Exception as e:
    print(f"An error occurred: {e}")

An error occurred: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH
